In [1]:
import pandas as pd
meta = pd.read_csv('data/pub_filter.csv')
rel = pd.read_csv('data/pub_relation.csv')
pub = pd.read_csv('data/dist_pub.csv')
per = pd.read_csv('data/dist_per.csv')

Upside down network

In [2]:
edge = pd.DataFrame()

In [3]:
edge_dict = {}
for i in range(0, len(meta)):
    if not meta['ODIS_PERS_ID'][i] in edge_dict:
        edge_dict[meta['ODIS_PERS_ID'][i]] = [meta['ODIS_ID'][i]]
    else:
        edge_dict[meta['ODIS_PERS_ID'][i]].append(meta['ODIS_ID'][i])

In [4]:
source = []
target = []
pub_id = []

for publication in edge_dict:
    people = list(set(edge_dict[publication]))
    pairs = [[a, b] for idx, a in enumerate(people) for b in people[idx + 1:]]
    for pair in pairs:
        source.append(pair[0])
        target.append(pair[1])
        pub_id.append(publication)

edge['Source'] = source
edge['Target'] = target
edge['ODIS_PERS_ID'] = pub_id

In [5]:
edge.shape

(461642, 3)

In [6]:
edge_merge = pd.merge(edge, per, on = 'ODIS_PERS_ID')

In [7]:
edge_merge.shape

(461642, 50)

In [8]:
edge_merge.head()

,Source,Target,ODIS_PERS_ID,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,Full Name,Birth Year,Birth Country,Birth Place,Death Year,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,35786,36812,119246,Smyth,Richard,Richard Smyth,1499.0,NaN,NaN,1563.0,...,0,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1560s
1,35786,35914,119246,Smyth,Richard,Richard Smyth,1499.0,NaN,NaN,1563.0,...,0,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1560s
2,36812,35914,119246,Smyth,Richard,Richard Smyth,1499.0,NaN,NaN,1563.0,...,0,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1560s
3,35786,35915,119215,Boscard,Jacques,Jacques Boscard,NaN,Belgium,Leuven,1580.0,...,0,Low Countries,Low Countries,Boscard,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,Multi
4,35786,35914,119215,Boscard,Jacques,Jacques Boscard,NaN,Belgium,Leuven,1580.0,...,0,Low Countries,Low Countries,Boscard,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,Multi


In [9]:
kellam_edge = edge_merge[edge_merge['Printing House'] == 'Kellam']

In [10]:
kellam_edge.shape

(12429, 50)

In [12]:
node = pub.rename(columns = {'ODIS_ID': 'id'})

In [13]:
kellam_id = [ID for ID in kellam_edge['Source']] + [ID for ID in kellam_edge['Target']]
kellam_id = list(set(kellam_id))

In [14]:
len(kellam_id)

260

In [15]:
kellam_node = node[node['id'].isin(kellam_id)]

In [16]:
non_empty_nodes = [ID for ID in kellam_edge['Source']] + [ID for ID in kellam_edge['Target']]
non_empty_nodes = list(set(non_empty_nodes))
empty_nodes = []
for ID in pub['ODIS_ID']:
    if not ID in non_empty_nodes:
        empty_nodes.append(ID)

In [17]:
# kellam_node.to_csv('sna/data/pub_node.csv', index = False)

Author published by the same printer/publisher

In [18]:
author_edge_dict = {}
for i in range(0, len(rel)):
    if rel['Relation Cluster'][i] == 'printer/publisher':
        temp_df = rel.loc[(rel['ODIS_ID'] == rel['ODIS_ID'][i]) & (rel['Relation Cluster'] == 'author')]
        if not rel['ODIS_PERS_ID'][i] in author_edge_dict:
            if not len([ID for ID in temp_df['ODIS_PERS_ID']]) == 0:
                author_edge_dict[rel['ODIS_PERS_ID'][i]] = list(set([ID for ID in temp_df['ODIS_PERS_ID']]))
        else:
            for ID in temp_df['ODIS_PERS_ID']:
                if not ID in author_edge_dict[rel['ODIS_PERS_ID'][i]]:
                    author_edge_dict[rel['ODIS_PERS_ID'][i]].append(ID)

In [19]:
author_edge = pd.DataFrame()

author_source = []
author_target = []
author_author_id = []

for pp in author_edge_dict:
    people = list(set(author_edge_dict[pp]))
    pairs = [[a, b] for idx, a in enumerate(people) for b in people[idx + 1:]]
    for pair in pairs:
        author_source.append(pair[0])
        author_target.append(pair[1])
        author_author_id.append(pp)

author_edge['Source'] = author_source
author_edge['Target'] = author_target
author_edge['ODIS_PERS_ID'] = author_author_id

In [20]:
author_edge.head()

,Source,Target,ODIS_PERS_ID
0,119746,119810,119215
1,119746,119853,119215
2,119746,119246,119215
3,119746,119671,119215
4,119746,119510,119215


In [21]:
author_edge.shape

(99219, 3)

In [22]:
author_edge_merge = pd.merge(author_edge, per, on = 'ODIS_PERS_ID')

In [23]:
author_edge_merge.shape

(99219, 50)

In [24]:
author_edge_k = author_edge_merge[author_edge_merge['Printing House'] == 'Kellam']

In [25]:
author_edge_k_id = list(set([ID for ID in author_edge_k['Source']] + [ID for ID in author_edge_k['Target']]))

In [28]:
author_nodes = [ID for ID in author_edge['Source']] + [ID for ID in author_edge['Target']]
author_nodes = list(set(author_nodes))

In [29]:
author_node = per.loc[per['ODIS_PERS_ID'].isin(author_nodes)]
author_node = author_node.rename(columns = {'ODIS_PERS_ID': 'id'})

In [30]:
author_node_k = per.loc[per['ODIS_PERS_ID'].isin(author_edge_k_id)]
author_node_k = author_node_k.rename(columns = {'ODIS_PERS_ID': 'id'})

Compiler/Editor/Revisor published by the same printer/publisher

In [33]:
cer_edge_dict = {}
for i in range(0, len(rel)):
    if rel['Relation Cluster'][i] == 'printer/publisher':
        temp_df = rel.loc[(rel['ODIS_ID'] == rel['ODIS_ID'][i]) & (rel['Relation Cluster'] == 'compiler/editor/revisor')]
        if not rel['ODIS_PERS_ID'][i] in cer_edge_dict:
            if not len([ID for ID in temp_df['ODIS_PERS_ID']]) == 0:
                cer_edge_dict[rel['ODIS_PERS_ID'][i]] = list(set([ID for ID in temp_df['ODIS_PERS_ID']]))
        else:
            for ID in temp_df['ODIS_PERS_ID']:
                if not ID in cer_edge_dict[rel['ODIS_PERS_ID'][i]]:
                    cer_edge_dict[rel['ODIS_PERS_ID'][i]].append(ID)

In [34]:
cer_edge = pd.DataFrame()

cer_source = []
cer_target = []
cer_cer_id = []

for pp in cer_edge_dict:
    people = list(set(cer_edge_dict[pp]))
    pairs = [[a, b] for idx, a in enumerate(people) for b in people[idx + 1:]]
    for pair in pairs:
        cer_source.append(pair[0])
        cer_target.append(pair[1])
        cer_cer_id.append(pp)

cer_edge['Source'] = cer_source
cer_edge['Target'] = cer_target
cer_edge['ODIS_PERS_ID'] = cer_cer_id

In [35]:
cer_edge.head()

,Source,Target,ODIS_PERS_ID
0,122624,123771,119268
1,122624,124677,119268
2,122624,123782,119268
3,122624,119930,119268
4,122624,122503,119268


In [36]:
cer_edge.shape

(2474, 3)

In [37]:
cer_edge_merge = pd.merge(cer_edge, per, on = 'ODIS_PERS_ID')

In [38]:
cer_edge_merge.shape

(2474, 50)

In [39]:
# cer_edge_merge.to_csv('sna/data/cer_edge.csv', index = False)

In [40]:
cer_nodes = [ID for ID in cer_edge['Source']] + [ID for ID in cer_edge['Target']]
cer_nodes = list(set(cer_nodes))

In [41]:
cer_node = per.loc[per['ODIS_PERS_ID'].isin(cer_nodes)]
cer_node = cer_node.rename(columns = {'ODIS_PERS_ID': 'id'})

In [42]:
# cer_node.to_csv('sna/data/cer_node.csv', index = False)

Printer/Publisher published the same author

In [43]:
pp_edge_dict = {}
for i in range(0, len(rel)):
    if rel['Relation Cluster'][i] == 'author':
        temp_df = rel.loc[(rel['ODIS_ID'] == rel['ODIS_ID'][i]) & (rel['Relation Cluster'] == 'printer/publisher')]
        if not rel['ODIS_PERS_ID'][i] in pp_edge_dict:
            if not len([ID for ID in temp_df['ODIS_PERS_ID']]) == 0:
                pp_edge_dict[rel['ODIS_PERS_ID'][i]] = list(set([ID for ID in temp_df['ODIS_PERS_ID']]))
        else:
            for ID in temp_df['ODIS_PERS_ID']:
                if not ID in pp_edge_dict[rel['ODIS_PERS_ID'][i]]:
                    pp_edge_dict[rel['ODIS_PERS_ID'][i]].append(ID)

In [44]:
pp_edge = pd.DataFrame()

pp_source = []
pp_target = []
pp_cer_id = []

for author in pp_edge_dict:
    people = list(set(pp_edge_dict[author]))
    pairs = [[a, b] for idx, a in enumerate(people) for b in people[idx + 1:]]
    for pair in pairs:
        pp_source.append(pair[0])
        pp_target.append(pair[1])
        pp_cer_id.append(author)

pp_edge['Source'] = pp_source
pp_edge['Target'] = pp_target
pp_edge['ODIS_PERS_ID'] = pp_cer_id

In [45]:
pp_edge.head()

,Source,Target,ODIS_PERS_ID
0,119602,121325,119246
1,119602,119215,119246
2,121325,119215,119246
3,119249,119268,119507
4,119249,119270,119507


In [46]:
pp_edge.shape

(1245, 3)

In [47]:
pp_edge_merge = pd.merge(pp_edge, per, on = 'ODIS_PERS_ID')

In [48]:
pp_edge_merge.shape

(1245, 50)

In [50]:
pp_nodes = [ID for ID in pp_edge['Source']] + [ID for ID in pp_edge['Target']]
pp_nodes = list(set(pp_nodes))

In [51]:
pp_node = per.loc[per['ODIS_PERS_ID'].isin(pp_nodes)]
pp_node = pp_node.rename(columns = {'ODIS_PERS_ID': 'id'})

In [53]:
import networkx as nx

Author

In [54]:
author_node = pd.read_csv('sna/data/author_node.csv')
author_edge = pd.read_csv('sna/data/author_edge.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_17100\4225978417.py:2: DtypeWarning: Columns (7,8,41) have mixed types. Specify dtype option on import or set low_memory=False.
  author_edge = pd.read_csv('sna/data/author_edge.csv')


In [55]:
author_cal = pd.read_csv('sna/data/author_cal.csv')

In [56]:
G_author = nx.Graph()
author_edge_list = [
        (author_edge['Source'][i], author_edge['Target'][i])
        for i in range(0, len(author_edge))
    ]
G_author.add_edges_from(author_edge_list)

In [57]:
author_degree_dict = nx.degree(G_author)
author_degree_cen_dict = nx.degree_centrality(G_author)
author_betweenness_dict = nx.betweenness_centrality(G_author)
author_eigenvector_dict = nx.eigenvector_centrality(G_author)
author_clustering_dict = nx.clustering(G_author)
author_closeness_dict = nx.closeness_centrality(G_author)

In [58]:
author_modularity_dict = author_cal[['Id', 'modularity_class']].rename(columns = {'Id': 'ODIS_PERS_ID'})

In [59]:
author_df = pd.DataFrame()
author_df['ODIS_PERS_ID'] = [ID for ID in per['ODIS_PERS_ID'] if ID in G_author.nodes]
author_df['Degree'] = [author_degree_dict[ID] for ID in author_df['ODIS_PERS_ID']]
author_df['Degree Centrality'] = [author_degree_cen_dict[ID] for ID in author_df['ODIS_PERS_ID']]
author_df['Eigenvector'] = [author_eigenvector_dict[ID] for ID in author_df['ODIS_PERS_ID']]
author_df['Clustering'] = [author_clustering_dict[ID] for ID in author_df['ODIS_PERS_ID']]
author_df['Closeness'] = [author_closeness_dict[ID] for ID in author_df['ODIS_PERS_ID']]
author_df['Betweenness'] = [author_betweenness_dict[ID] for ID in author_df['ODIS_PERS_ID']]
author_df = pd.merge(author_df, author_modularity_dict, on = 'ODIS_PERS_ID')
author_df = pd.merge(author_df, per, on = 'ODIS_PERS_ID')

In [60]:
# author_df.to_csv('author_degrees.csv', index = False)

In [61]:
sort_degree_df = author_df.sort_values('Degree', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119703,781,0.658516,0.068042,0.260629,0.740832,0.099001,3,a Kempis,Thomas,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
1,119595,641,0.540472,0.065516,0.305694,0.679638,0.049698,0,de Granada,Luis,...,0,Iberian Union,Iberian Union,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s
2,120372,625,0.526981,0.066134,0.325308,0.673327,0.024149,0,of Clairvaux,Bernard,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1630s
3,119699,584,0.492411,0.063450,0.405349,0.652132,0.041101,2,Carthusianus,Dionysius,...,0,Low Countries,NaN,NaN,NaN,NaN,Male or Unspecified,author,Multi,Multi
4,119689,583,0.491568,0.064117,0.349024,0.658050,0.033784,3,de Blois,Louis,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s


In [62]:
sort_degree_df = author_df.sort_values('Eigenvector', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119703,781,0.658516,0.068042,0.260629,0.740832,0.099001,3,a Kempis,Thomas,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
1,120372,625,0.526981,0.066134,0.325308,0.673327,0.024149,0,of Clairvaux,Bernard,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1630s
2,119595,641,0.540472,0.065516,0.305694,0.679638,0.049698,0,de Granada,Luis,...,0,Iberian Union,Iberian Union,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s
3,119629,484,0.408094,0.064347,0.475908,0.622394,0.004373,5,of Hippo,Augustine,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
4,119689,583,0.491568,0.064117,0.349024,0.658050,0.033784,3,de Blois,Louis,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s


In [63]:
sort_degree_df = author_df.sort_values('Clustering', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,53546,135,0.113828,0.013653,1.0,0.513337,0.0,3,de Loyola,Ignatius,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1600s
1,124206,45,0.037943,0.006151,1.0,0.488893,0.0,5,Henot,Eberhard,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
2,124168,89,0.075042,0.006760,1.0,0.510626,0.0,0,de Saint-Laurens,Jean,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
3,124170,198,0.166948,0.002956,1.0,0.486637,0.0,2,Struellens,André,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
4,124171,89,0.075042,0.006760,1.0,0.510626,0.0,0,Rubéric,Severin,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1620s


In [64]:
sort_degree_df = author_df.sort_values('Closeness', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119703,781,0.658516,0.068042,0.260629,0.740832,0.099001,3,a Kempis,Thomas,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
1,119595,641,0.540472,0.065516,0.305694,0.679638,0.049698,0,de Granada,Luis,...,0,Iberian Union,Iberian Union,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s
2,120372,625,0.526981,0.066134,0.325308,0.673327,0.024149,0,of Clairvaux,Bernard,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1630s
3,119689,583,0.491568,0.064117,0.349024,0.658050,0.033784,3,de Blois,Louis,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s
4,119699,584,0.492411,0.063450,0.405349,0.652132,0.041101,2,Carthusianus,Dionysius,...,0,Low Countries,NaN,NaN,NaN,NaN,Male or Unspecified,author,Multi,Multi


In [65]:
sort_degree_df = author_df.sort_values('Betweenness', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119703,781,0.658516,0.068042,0.260629,0.740832,0.099001,3,a Kempis,Thomas,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1610s
1,119595,641,0.540472,0.065516,0.305694,0.679638,0.049698,0,de Granada,Luis,...,0,Iberian Union,Iberian Union,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s
2,119699,584,0.492411,0.063450,0.405349,0.652132,0.041101,2,Carthusianus,Dionysius,...,0,Low Countries,NaN,NaN,NaN,NaN,Male or Unspecified,author,Multi,Multi
3,122794,555,0.467960,0.060633,0.415214,0.646679,0.038900,2,Trittheim,Johann,...,0,Holy Roman Empire,Holy Roman Empire,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1600s
4,119689,583,0.491568,0.064117,0.349024,0.658050,0.033784,3,de Blois,Louis,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1590s


Cer

In [66]:
cer_node = pd.read_csv('sna/data/cer_node.csv')
cer_edge = pd.read_csv('sna/data/cer_edge.csv')

In [67]:
cer_cal = pd.read_csv('sna/data/cer_cal.csv')

In [68]:
G_cer = nx.Graph()
cer_edge_list = [
        (cer_edge['Source'][i], cer_edge['Target'][i])
        for i in range(0, len(cer_edge))
    ]
G_cer.add_edges_from(cer_edge_list)

In [69]:
cer_degree_dict = nx.degree(G_cer)
cer_degree_cen_dict = nx.degree_centrality(G_cer)
cer_betweenness_dict = nx.betweenness_centrality(G_cer)
cer_eigenvector_dict = nx.eigenvector_centrality(G_cer)
cer_clustering_dict = nx.clustering(G_cer)
cer_closeness_dict = nx.closeness_centrality(G_cer)

In [70]:
cer_modularity_dict = cer_cal[['Id', 'modularity_class']].rename(columns = {'Id': 'ODIS_PERS_ID'})

In [71]:
cer_df = pd.DataFrame()
cer_df['ODIS_PERS_ID'] = [ID for ID in per['ODIS_PERS_ID'] if ID in G_cer.nodes]
cer_df['Degree'] = [cer_degree_dict[ID] for ID in cer_df['ODIS_PERS_ID']]
cer_df['Degree Centrality'] = [cer_degree_cen_dict[ID] for ID in cer_df['ODIS_PERS_ID']]
cer_df['Eigenvector'] = [cer_eigenvector_dict[ID] for ID in cer_df['ODIS_PERS_ID']]
cer_df['Clustering'] = [cer_clustering_dict[ID] for ID in cer_df['ODIS_PERS_ID']]
cer_df['Closeness'] = [cer_closeness_dict[ID] for ID in cer_df['ODIS_PERS_ID']]
cer_df['Betweenness'] = [cer_betweenness_dict[ID] for ID in cer_df['ODIS_PERS_ID']]
cer_df = pd.merge(cer_df, cer_modularity_dict, on = 'ODIS_PERS_ID')
cer_df = pd.merge(cer_df, per, on = 'ODIS_PERS_ID')

In [72]:
# cer_df.to_csv('cer_degrees.csv', index = False)

In [73]:
sort_degree_df = cer_df.sort_values('Degree', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,120271,95,0.753968,0.134069,0.454871,0.764463,0.096943,2,Gibbons,Richard,...,1,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
1,119785,93,0.738095,0.134454,0.484572,0.779855,0.084060,0,Colveneere,Georges,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,approbator,FRENCH,1610s
2,122650,92,0.730159,0.133762,0.488772,0.774656,0.106935,0,de Sommal,Henri,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
3,123361,89,0.706349,0.132142,0.507661,0.759467,0.100173,0,Sylvius,François,...,1,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1620s
4,119625,78,0.619048,0.131225,0.651349,0.704233,0.029542,0,de Leuze,Nicolas,...,0,Low Countries,NaN,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,FRENCH,Multi


In [74]:
sort_degree_df['Full Name'].head(26)

0           Richard Gibbons
1        Georges Colveneere
2           Henri de Sommal
3          François Sylvius
4          Nicolas de Leuze
5                John Jones
6            Matthäus Rader
7           Baudouin Willot
8            Jean Le Maieur
9               John Barnes
10      Gonzalo de Barnuevo
11       Barthélémy Peeters
12             Richard Hall
13        Andrea Victorelli
14          Guillaume Gazet
15          Adrianus Junius
16        Antoine Lespagnol
17           Géry Lespagnol
18    Maximilien Thieulaine
19             Luke Wadding
20        Hendrik Van Cuyck
21        Richard Hudleston
22              Alard Gazet
23        Pierre d'Outreman
24          Horatius Lutius
25            Jean Soteaulx
Name: Full Name, dtype: object

In [75]:
sort_degree_df = cer_df.sort_values('Eigenvector', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119785,93,0.738095,0.134454,0.484572,0.779855,0.084060,0,Colveneere,Georges,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,approbator,FRENCH,1610s
1,120271,95,0.753968,0.134069,0.454871,0.764463,0.096943,2,Gibbons,Richard,...,1,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
2,122650,92,0.730159,0.133762,0.488772,0.774656,0.106935,0,de Sommal,Henri,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
3,123361,89,0.706349,0.132142,0.507661,0.759467,0.100173,0,Sylvius,François,...,1,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1620s
4,119625,78,0.619048,0.131225,0.651349,0.704233,0.029542,0,de Leuze,Nicolas,...,0,Low Countries,NaN,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,FRENCH,Multi


In [76]:
sort_degree_df = cer_df.sort_values('Clustering', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,116703,2,0.015873,0.002294,1.0,0.427200,0.0,0,Puteanus,Erycius,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1620s
1,124256,60,0.476190,0.126166,1.0,0.618077,0.0,3,Tossinianensis,Petrus,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
2,124839,9,0.071429,0.017397,1.0,0.482151,0.0,2,d'Espières,Jean,...,4,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,LATIN,Multi
3,124740,3,0.023810,0.006284,1.0,0.389928,0.0,3,Petit,Philippe,...,3,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,author,FRENCH,1650s
4,124701,9,0.071429,0.017397,1.0,0.482151,0.0,2,White,William,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1630s


In [77]:
sort_degree_df = cer_df.sort_values('Closeness', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119785,93,0.738095,0.134454,0.484572,0.779855,0.084060,0,Colveneere,Georges,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,approbator,FRENCH,1610s
1,122650,92,0.730159,0.133762,0.488772,0.774656,0.106935,0,de Sommal,Henri,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
2,120271,95,0.753968,0.134069,0.454871,0.764463,0.096943,2,Gibbons,Richard,...,1,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
3,123361,89,0.706349,0.132142,0.507661,0.759467,0.100173,0,Sylvius,François,...,1,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1620s
4,119625,78,0.619048,0.131225,0.651349,0.704233,0.029542,0,de Leuze,Nicolas,...,0,Low Countries,NaN,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,FRENCH,Multi


In [78]:
sort_degree_df = cer_df.sort_values('Betweenness', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,122650,92,0.730159,0.133762,0.488772,0.774656,0.106935,0,de Sommal,Henri,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
1,123361,89,0.706349,0.132142,0.507661,0.759467,0.100173,0,Sylvius,François,...,1,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,author,LATIN,1620s
2,120271,95,0.753968,0.134069,0.454871,0.764463,0.096943,2,Gibbons,Richard,...,1,NaN,Low Countries,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,1610s
3,119785,93,0.738095,0.134454,0.484572,0.779855,0.084060,0,Colveneere,Georges,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,approbator,FRENCH,1610s
4,138454,21,0.166667,0.015945,0.619048,0.516437,0.073651,0,Pighius,Stephanus Wijnand,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,compiler/editor/revisor,LATIN,Multi


PriPub

In [79]:
pp_node = pd.read_csv('sna/data/pp_node.csv')
pp_edge = pd.read_csv('sna/data/pp_edge.csv')

In [80]:
pp_cal = pd.read_csv('sna/data/pp_cal.csv')

In [81]:
G_pp = nx.Graph()
pp_edge_list = [
        (pp_edge['Source'][i], pp_edge['Target'][i])
        for i in range(0, len(pp_edge))
    ]
G_pp.add_edges_from(pp_edge_list)

In [82]:
pp_degree_dict = nx.degree(G_pp)
pp_degree_cen_dict = nx.degree_centrality(G_pp)
pp_betweenness_dict = nx.betweenness_centrality(G_pp)
pp_eigenvector_dict = nx.eigenvector_centrality(G_pp)
pp_clustering_dict = nx.clustering(G_pp)
pp_closeness_dict = nx.closeness_centrality(G_pp)

In [83]:
pp_modularity_dict = pp_cal[['Id', 'modularity_class']].rename(columns = {'Id': 'ODIS_PERS_ID'})

In [84]:
pp_df = pd.DataFrame()
pp_df['ODIS_PERS_ID'] = [ID for ID in per['ODIS_PERS_ID'] if ID in G_pp.nodes]
pp_df['Degree'] = [pp_degree_dict[ID] for ID in pp_df['ODIS_PERS_ID']]
pp_df['Degree Centrality'] = [pp_degree_cen_dict[ID] for ID in pp_df['ODIS_PERS_ID']]
pp_df['Eigenvector'] = [pp_eigenvector_dict[ID] for ID in pp_df['ODIS_PERS_ID']]
pp_df['Clustering'] = [pp_clustering_dict[ID] for ID in pp_df['ODIS_PERS_ID']]
pp_df['Closeness'] = [pp_closeness_dict[ID] for ID in pp_df['ODIS_PERS_ID']]
pp_df['Betweenness'] = [pp_betweenness_dict[ID] for ID in pp_df['ODIS_PERS_ID']]
pp_df = pd.merge(pp_df, pp_modularity_dict, on = 'ODIS_PERS_ID')
pp_df = pd.merge(pp_df, per, on = 'ODIS_PERS_ID')

In [85]:
# pp_df.to_csv('pp_degrees.csv', index = False)

In [86]:
sort_degree_df = pp_df.sort_values('Degree', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119268,59,0.728395,0.279430,0.199883,0.786408,0.342845,1,Bellère,Balthazar I,...,1,Low Countries,Low Countries,Bellère,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
1,119249,41,0.506173,0.241456,0.335366,0.658537,0.118240,1,Bogart,Jan I,...,0,NaN,Low Countries,Bogart,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1590s
2,119288,39,0.481481,0.246783,0.392713,0.637795,0.067153,1,Auroy,Pierre,...,0,Low Countries,Low Countries,Auroy,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
3,119285,33,0.407407,0.224127,0.460227,0.609023,0.035562,1,Kellam,Laurence II,...,16,Low Countries,Low Countries,Kellam,Britain,NaN,Male or Unspecified,printer/publisher,LATIN,1620s
4,119283,33,0.407407,0.217829,0.429924,0.609023,0.045929,2,Lanseart,Marguerite,...,0,NaN,NaN,Kellam,Britain,NaN,Female Pri/Pub,printer/publisher,LATIN,1610s


In [87]:
pp_df.sort_values('Degree', ascending = False).reset_index(drop = True)['Full Name'].head(29)

0           Balthazar I Bellère
1                  Jan I Bogart
2                  Pierre Auroy
3            Laurence II Kellam
4           Marguerite Lanseart
5                Jean II Bogart
6             Laurence I Kellam
7            Pierre I Borremans
8                     Marc Wyon
9             Charles I Boscard
10               Gérard Pinchon
11             Marie Vanderpiet
12              Jean de Fampoux
13                 Gérard Patté
14                 John Heigham
15                Martin Bogart
16               Jean Serrurier
17         Balthazar II Bellere
18              Jeanne de Menin
19         Christine De Roovere
20                Pierre Bogart
21               Gertrud Heusch
22             Marie Borrewater
23    Guillaume I de La Rivière
24            Barthélémy Bardou
25            Thomas (I) Kellam
26             Philippus Nutius
27             Dorothée Boscard
28                Jean de Spira
Name: Full Name, dtype: object

In [88]:
sort_degree_df = pp_df.sort_values('Eigenvector', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119268,59,0.728395,0.279430,0.199883,0.786408,0.342845,1,Bellère,Balthazar I,...,1,Low Countries,Low Countries,Bellère,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
1,119288,39,0.481481,0.246783,0.392713,0.637795,0.067153,1,Auroy,Pierre,...,0,Low Countries,Low Countries,Auroy,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
2,119249,41,0.506173,0.241456,0.335366,0.658537,0.118240,1,Bogart,Jan I,...,0,NaN,Low Countries,Bogart,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1590s
3,119285,33,0.407407,0.224127,0.460227,0.609023,0.035562,1,Kellam,Laurence II,...,16,Low Countries,Low Countries,Kellam,Britain,NaN,Male or Unspecified,printer/publisher,LATIN,1620s
4,119283,33,0.407407,0.217829,0.429924,0.609023,0.045929,2,Lanseart,Marguerite,...,0,NaN,NaN,Kellam,Britain,NaN,Female Pri/Pub,printer/publisher,LATIN,1610s


In [89]:
sort_degree_df = pp_df.sort_values('Clustering', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119338,4,0.049383,0.024291,1.0,0.413265,0.0,0,de Buyens,Claude,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,printer/publisher,FRENCH,1570s
1,119388,8,0.098765,0.074081,1.0,0.487952,0.0,1,de Rache,Pierre,...,0,Low Countries,Low Countries,NaN,NaN,NaN,Male or Unspecified,printer/publisher,FRENCH,1610s
2,123595,2,0.024691,0.014812,1.0,0.445055,0.0,0,Velpius,Reinerus II,...,0,NaN,NaN,Velpius,Low Countries,NaN,Male or Unspecified,printer/publisher,LATIN,1600s
3,119316,10,0.123457,0.089077,1.0,0.493902,0.0,4,Taylor,Henry,...,0,NaN,NaN,Henry,Britain,founder,Male or Unspecified,printer/publisher,ENGLISH,1620s
4,119317,4,0.049383,0.033527,1.0,0.455056,0.0,3,Doby,Antoine,...,0,NaN,NaN,NaN,NaN,NaN,Male or Unspecified,printer/publisher,Multi,1620s


In [90]:
sort_degree_df = pp_df.sort_values('Closeness', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119268,59,0.728395,0.279430,0.199883,0.786408,0.342845,1,Bellère,Balthazar I,...,1,Low Countries,Low Countries,Bellère,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
1,119249,41,0.506173,0.241456,0.335366,0.658537,0.118240,1,Bogart,Jan I,...,0,NaN,Low Countries,Bogart,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1590s
2,119288,39,0.481481,0.246783,0.392713,0.637795,0.067153,1,Auroy,Pierre,...,0,Low Countries,Low Countries,Auroy,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
3,119285,33,0.407407,0.224127,0.460227,0.609023,0.035562,1,Kellam,Laurence II,...,16,Low Countries,Low Countries,Kellam,Britain,NaN,Male or Unspecified,printer/publisher,LATIN,1620s
4,119283,33,0.407407,0.217829,0.429924,0.609023,0.045929,2,Lanseart,Marguerite,...,0,NaN,NaN,Kellam,Britain,NaN,Female Pri/Pub,printer/publisher,LATIN,1610s


In [91]:
sort_degree_df = pp_df.sort_values('Betweenness', ascending = False).reset_index(drop = True)
sort_degree_df.head()

,ODIS_PERS_ID,Degree,Degree Centrality,Eigenvector,Clustering,Closeness,Betweenness,modularity_class,REL_PERS_REF_SURNAME,REL_PERS_REF_FIRST_NAME,...,1650s,Birth Region,Death Region,Printing House,House Origion,House Note,Gender,Most Involved Role,Most Involved Language,Most Involved Period
0,119268,59,0.728395,0.279430,0.199883,0.786408,0.342845,1,Bellère,Balthazar I,...,1,Low Countries,Low Countries,Bellère,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
1,119249,41,0.506173,0.241456,0.335366,0.658537,0.118240,1,Bogart,Jan I,...,0,NaN,Low Countries,Bogart,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1590s
2,119303,26,0.320988,0.190098,0.538462,0.574468,0.078043,1,Wyon,Marc,...,0,Low Countries,Low Countries,Wyon,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1610s
3,119288,39,0.481481,0.246783,0.392713,0.637795,0.067153,1,Auroy,Pierre,...,0,Low Countries,Low Countries,Auroy,Low Countries,founder,Male or Unspecified,printer/publisher,LATIN,1620s
4,119252,31,0.382716,0.214136,0.470968,0.600000,0.054506,1,Bogart,Jean II,...,0,Low Countries,Low Countries,Bogart,Low Countries,NaN,Male or Unspecified,printer/publisher,LATIN,1620s
